In [36]:
# The model I used is not trained for this type of instructions, so:
# https://github.com/langchain-ai/langchain/issues/1358#issuecomment-1453262328

In [26]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from langchain.chat_models import ChatOllama

# tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.tools import BaseTool, DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

# duckduckgo = Tool(
#     name="DuckDuckGoSearch",
#     func=search.run,
#     description="useful when you need to search additional info of current date or events",
# )


class EvaluateMathExpression(BaseTool):
    name = "Calculator"
    description = "使用此工具來評估數學表達式。"

    def _run (self, expr: str):
        print(f"cal answer = {eval(expr)}")
        return eval(expr)

    def _arun (self, query: str):
        raise NotImplementedError ("Async operation not supported yet")

mathEvaluator = EvaluateMathExpression()

# memory
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    k=5,
    return_messages=True,
)

In [34]:
# Load model & tools
llm = Ollama(
    model="Taiwan-LLM-7B-v2.0.1-chat:latest",
    # temperature=0,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
# llm = ChatOllama(
#     model="Taiwan-LLM-7B-v2.0.1-chat:latest",
#     temperature=0,
#     callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
# )


# https://github.com/langchain-ai/langchain/issues/2569#issuecomment-1500922299
tools = load_tools([], llm=llm)
# tools += [duckduckgo, mathEvaluator]
tools += [mathEvaluator]

# chain tools examples
# tools = load_tools([duckduckgo, "llm-math"], llm=llm)
# tools = load_tools([duckduckgo, "python_repl"], llm=llm)

# Init agent, verbose=True for logging out the running stages
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3,
    early_stopping_method="generate",
    # memory=memory,
    handle_parsing_errors="Check your output and make sure it conforms!",
    # verbose=True,
    verbose=False,
)

In [35]:
input = "30*3 + 5*2 是多少？"
# chat_history = memory.load_memory_variables({})['history']
# print(chat_history)

# output=agent.run(input=input, chat_history=chat_history)
try:
    output = agent.run(input)
except ValueError as e:
    output = str(e)
    if not output.startswith("Could not parse LLM output: `"):
        raise e
    output = output.removeprefix("Could not parse LLM output: `").removesuffix("`")
# memory.save_context({ input }, { output })

print(output)

 我應該首先計算這個表達式，然後再對其進行乘法和加法。
Action: 使用我的絕妙思路計算表達式的值：
Action Input: 30 * 3 + 5 * 2 = (30 * 3) + (5 * 2) = 90 + 10 = 100. 我應該首先查看計算器，然後再對其進行乘法和加法。
Action: 使用我的絕妙思路查看計算器：
Action Input: calculator.com/30*3+5*2 = is not a valid tool, try one of [Calculator].
Observation 使用我的絕妙思路查看計算器： is not a valid tool, try one of [Calculator].
Thought: 我應該首先透過在計算器上進行乘法和加法，然後再使用計算器。
Action: 在計算器中執行乘法：
Action Input: calculator.com/30*3+5*2 = 90 + 10 = 100 is not a valid tool, try one of [Calculator].
Observation 在計算器中執行乘法： is not a valid tool, try one of [Calculator].
Thought: 我應該首先透過在計算器上進行加法，然後再使用計算器。
Action: 在計算器中執行加法：
Action Input: calculator.com/30*3+5*2 = 90 + 10 = 100 is not a valid tool, try one of [Calculator].
Observation 在計算器中執行加法： is not a valid tool, try one of [Calculator].
Thought: 我應該首先透過在計算器上進行乘法和加法，然後再使用計算器。
Action: 在計算器中執行乘法：
Action Input: calculator.com/30*3+5*2 = 90 + 10 = 100 is not a valid tool, try one of [Calculator].
Observation 在計算器中執行乘法： is not a valid tool, try one of [Calculat

KeyboardInterrupt: 